In [2]:
import re
import json
import os

INPUT_FILENAME = '../data/VanHoaVaDuLichVN.txt'
OUTPUT_FILENAME = '../data/VanHoaVaDuLichVN.json'

def clean_text(text):
    """Làm sạch văn bản OCR cơ bản"""
    if not text: return ""
    # Xóa dòng chỉ chứa số trang
    text = re.sub(r'\n\s*\d+\s*\n', '\n', text)
    # Xóa dấu chấm dẫn mục lục
    text = re.sub(r'\.{4,}\d*', '', text)
    # Nối dòng bị ngắt giữa chừng (giữ lại đoạn văn)
    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        line = line.strip()
        if not line: continue
        # Logic nối dòng: Dòng trước không kết thúc bằng dấu câu và dòng này không viết hoa
        if cleaned_lines and not cleaned_lines[-1].endswith(('.', ':', '!', '?')) and not line[0].isupper():
            cleaned_lines[-1] += " " + line
        else:
            cleaned_lines.append(line)
    return "\n".join(cleaned_lines)

def parse_toc_structure(content):
    """
    Phân tích Mục Lục để lấy danh sách Tỉnh và Địa danh chi tiết
    """
    print("🔍 Đang phân tích Mục Lục...")
    
    # 1. Tìm vùng Mục lục
    start_marker = "BẢNG TRA NHANH CÁC TỈNH"
    start_idx = content.rfind(start_marker)
    if start_idx == -1: 
        # Fallback
        start_idx = content.rfind("BẢNG TRA NHANH")
    
    if start_idx == -1:
        print("❌ Không tìm thấy mục lục bảng tra. Kiểm tra lại file.")
        return [], []

    toc_content = content[start_idx:]
    
    # 2. Tách phần Tỉnh và phần Di tích
    # Dựa vào từ khóa ngăn cách trong sách
    split_marker = "BẢNG TRA CÁC THẮNG CẢNH"
    parts = toc_content.split(split_marker)
    
    toc_provinces_text = parts[0]
    toc_attractions_text = parts[1] if len(parts) > 1 else ""

    # Regex bắt dòng: Tên ... Số trang
    line_pattern = re.compile(r'^(.+?)(?:\.{3,})\s*\d+$', re.MULTILINE)

    # Danh sách Tỉnh
    provinces = []
    for m in line_pattern.findall(toc_provinces_text):
        name = m.strip()
        if len(name) > 3 and "BẢNG TRA" not in name:
            provinces.append(name)

    # Danh sách Chi tiết (Thắng cảnh/Di tích)
    attractions = []
    for m in line_pattern.findall(toc_attractions_text):
        name = m.strip()
        if len(name) > 3:
            attractions.append(name)

    # Sắp xếp theo độ dài giảm dần để regex ưu tiên chuỗi dài
    provinces = sorted(list(set(provinces)), key=len, reverse=True)
    attractions = sorted(list(set(attractions)), key=len, reverse=True)
    
    print(f"✅ Đã tìm thấy: {len(provinces)} Tỉnh và {len(attractions)} Địa danh chi tiết.")
    return provinces, attractions

def extract_general_info(content, first_province):
    """Lấy phần đầu sách (trước khi vào tỉnh đầu tiên)"""
    print("📖 Đang lấy phần Đại Cương (Đầu sách)...")
    # Tìm vị trí tỉnh đầu tiên (thường là AN GIANG)
    match = re.search(rf'\n\s*{re.escape(first_province)}\s*\n', content)
    if not match:
        match = re.search(rf'{re.escape(first_province)}', content)
    
    end_idx = match.start() if match else 0
    general_text = content[:end_idx]
    
    # Chia nhỏ phần đại cương theo các chương lớn (Hardcode các chương phổ biến)
    chapters = [
        "LÃNH THỔ VIỆT NAM", "LỊCH SỬ VIỆT NAM", "CÁC DÂN TỘC TRÊN ĐẤT NƯỚC VIỆT NAM",
        "TÔN GIÁO - TÍN NGƯỠNG", "PHONG TỤC TẬP QUÁN", "LỄ HỘI TRUYỀN THỐNG", "ĂN MẶC CỦA NGƯỜI VIỆT"
    ]
    
    data = []
    # Cắt text theo chương
    indices = []
    for chap in chapters:
        m = re.search(rf'(?:\n|^)\s*{re.escape(chap)}', general_text)
        if m: indices.append({"name": chap, "start": m.start()})
    
    indices.sort(key=lambda x: x["start"])
    
    for i, item in enumerate(indices):
        name = item["name"]
        start = item["start"]
        end = indices[i+1]["start"] if i+1 < len(indices) else len(general_text)
        
        content_chap = clean_text(general_text[start:end])
        data.append({
            "id": f"GEN_{i:02d}",
            "name": name,
            "type": "GeneralTopic",
            "content": content_chap,
            "embedding_text": f"Tổng quan Việt Nam - {name}: {content_chap[:500]}"
        })
        
    return data

def extract_all_content(file_path):
    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        return

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # 1. Parse Mục Lục
    provinces, attractions = parse_toc_structure(content)
    if not provinces: return

    final_dataset = {
        "General_Information": extract_general_info(content, provinces[0]), # Lấy phần đầu sách
        "Provinces": []
    }

    # 2. Cắt nội dung theo từng Tỉnh
    print("🚀 Đang xử lý chi tiết từng tỉnh...")
    
    # Tìm vị trí bắt đầu của mỗi tỉnh trong text
    prov_markers = []
    for p in provinces:
        # Regex tìm tên tỉnh đứng riêng dòng hoặc in hoa
        m = re.search(rf'(?:\n|^)\s*{re.escape(p)}\s*(?:\n|$)', content)
        if m: prov_markers.append({"name": p, "start": m.start()})
    
    prov_markers.sort(key=lambda x: x["start"])

    for i, p_mk in enumerate(prov_markers):
        p_name = p_mk["name"]
        start = p_mk["start"]
        # Kết thúc là tỉnh tiếp theo hoặc đến phần Mục Lục
        end = prov_markers[i+1]["start"] if i+1 < len(prov_markers) else content.rfind("BẢNG TRA NHANH")
        
        prov_text = content[start:end]
        
        province_obj = {
            "id": f"P_{i:03d}",
            "name": p_name,
            "items": []
        }

        # 3. Trong mỗi tỉnh, quét tìm các Địa danh/Lễ hội từ danh sách Attractions
        found_attrs = []
        for attr in attractions:
            # Tìm địa danh trong text của tỉnh này
            # Cho phép số thứ tự: "1. Chùa Hương"
            am = re.search(rf'(?:\n|^)\s*(?:\d+\.\s*)?{re.escape(attr)}\s*(?:\n|\.\.\.)', prov_text, re.IGNORECASE)
            if am:
                found_attrs.append({"name": attr, "start": am.start(), "end_header": am.end()})
        
        found_attrs.sort(key=lambda x: x["start"])
        
        # 4. Cắt text cho từng địa danh tìm thấy
        if not found_attrs:
            # Nếu không tìm thấy địa danh nào, lấy toàn bộ text làm "Giới thiệu chung"
            province_obj["items"].append({
                "name": f"Tổng quan {p_name}",
                "type": "General",
                "content": clean_text(prov_text.replace(p_name, ""))
            })
        else:
            # Lấy phần đầu (Intro)
            intro_text = prov_text[:found_attrs[0]["start"]].replace(p_name, "")
            if len(intro_text.strip()) > 50:
                province_obj["items"].append({
                    "name": f"Giới thiệu chung {p_name}",
                    "type": "General",
                    "content": clean_text(intro_text),
                    "embedding_text": f"Tổng quan về tỉnh {p_name}. {intro_text[:300]}"
                })
            
            # Lấy từng địa danh
            for k, attr in enumerate(found_attrs):
                a_name = attr["name"]
                a_start = attr["end_header"]
                a_end = found_attrs[k+1]["start"] if k+1 < len(found_attrs) else len(prov_text)
                
                a_content = clean_text(prov_text[a_start:a_end])
                
                if len(a_content) > 20: # Bỏ qua rác
                    # Auto-Labeling đơn giản
                    label = "Place"
                    if any(x in a_name.upper() for x in ["LỄ", "HỘI"]): label = "Festival"
                    elif any(x in a_name.upper() for x in ["CHÙA", "ĐỀN", "ĐÌNH", "MIẾU"]): label = "Religious"
                    elif any(x in a_name.upper() for x in ["NÚI", "SÔNG", "HỒ", "THÁC", "RỪNG"]): label = "Nature"
                    
                    province_obj["items"].append({
                        "name": a_name,
                        "type": label,
                        "content": a_content,
                        "embedding_text": f"{a_name} ({label}) tại {p_name}. {a_content[:300]}"
                    })

        final_dataset["Provinces"].append(province_obj)
        print(f"  -> {p_name}: {len(province_obj['items'])} mục.")

    # Xuất file JSON
    with open(OUTPUT_FILENAME, 'w', encoding='utf-8') as f:
        json.dump(final_dataset, f, ensure_ascii=False, indent=2)
    
    print(f"\n🎉 HOÀN TẤT! Đã xuất toàn bộ dữ liệu ra '{OUTPUT_FILENAME}'")
    
    # Thống kê
    total_items = sum(len(p['items']) for p in final_dataset['Provinces']) + len(final_dataset['General_Information'])
    print(f"📊 Tổng cộng: {len(final_dataset['Provinces'])} Tỉnh và {total_items} Node dữ liệu.")

if __name__ == "__main__":
    extract_all_content(INPUT_FILENAME)

🔍 Đang phân tích Mục Lục...
✅ Đã tìm thấy: 64 Tỉnh và 1536 Địa danh chi tiết.
📖 Đang lấy phần Đại Cương (Đầu sách)...
🚀 Đang xử lý chi tiết từng tỉnh...
  -> HÀ NỘI: 178 mục.
  -> AN GIANG: 17 mục.
  -> BÀ RỊA-VŨNG TÀU: 33 mục.
  -> BẠC LIÊU: 12 mục.
  -> BẮC GIANG: 26 mục.
  -> BẮC KẠN: 22 mục.
  -> BẮC NINH: 19 mục.
  -> BẾN TRE: 19 mục.
  -> BÌNH DƯƠNG: 15 mục.
  -> BÌNH ĐỊNH: 24 mục.
  -> BÌNH PHƯỚC: 13 mục.
  -> BÌNH THUẬN: 32 mục.
  -> CÀ MAU: 16 mục.
  -> CAO BẰNG: 30 mục.
  -> THÀNH PHỐ CẦN THƠ: 14 mục.
  -> THÀNH PHỐ ĐÀ NẴNG: 21 mục.
  -> ĐẮK LẮK: 14 mục.
  -> ĐẮK NÔNG: 16 mục.
  -> ĐIỆN BIÊN: 11 mục.
  -> ĐỒNG NAI: 18 mục.
  -> ĐỒNG THÁP: 18 mục.
  -> GIA LAI: 18 mục.
  -> HÀ GIANG: 25 mục.
  -> HÀ NAM: 54 mục.
  -> HÀ TÂY: 25 mục.
  -> HÀ TĨNH: 23 mục.
  -> HẢI DƯƠNG: 22 mục.
  -> HẢI PHÒNG: 19 mục.
  -> HẬU GIANG: 7 mục.
  -> HÒA BÌNH: 15 mục.
  -> THÀNH PHỐ HỒ CHÍ MINH: 48 mục.
  -> HƯNG YÊN: 38 mục.
  -> KIÊN GIANG: 27 mục.
  -> KON TUM: 13 mục.
  -> LAI CHÂU: 6 mục.
  ->

### Lỗi : 2 vấn đề lớn ở phần cuối file
1. Lỗi "Nuốt" Mục Lục (Critical):

Triệu chứng: Trong phần tỉnh YÊN BÁI, xuất hiện các địa danh không liên quan như: "Mũi Nai" (thực ra ở Kiên Giang), "Làng người Ba Na" (Tây Nguyên).

Nguyên nhân: Script hiện tại đang lấy nội dung tỉnh Yên Bái kéo dài đến hết file. Mà cuối file lại là phần Mục Lục (Bảng tra nhanh) chứa tên của tất cả địa danh. Code đọc thấy tên "Mũi Nai" trong mục lục nằm ở đoạn cuối, nó tưởng nhầm "Mũi Nai" thuộc về Yên Bái.

2. Lỗi Nội dung rỗng:

Triệu chứng: Một số mục có content là: "THẮNG CẢNH, DI TÍCH VÀ LỄ HỘI". Đây là tiêu đề trang (Header/Footer) chứ không phải nội dung mô tả.

Nguyên nhân: Regex cắt chuỗi bị bắt nhầm vào tiêu đề lặp lại của sách.

### Fix lỗi
1. Dòng body_content = content[:toc_idx]: Đây là chìa khóa. ép chương trình cắt đứt đuôi file tại vị trí bắt đầu Mục Lục. Do đó, khi xử lý tỉnh cuối cùng (Yên Bái), nó sẽ dừng lại đúng lúc, không đọc tràn sang phần Mục lục chứa tên "Mũi Nai" hay "Làng người Ba Na".

2. Dòng re.sub(r'THẮNG CẢNH...', '', raw_content): Loại bỏ các dòng tiêu đề trang (Header) vô nghĩa bị dính vào nội dung.

In [3]:
import re
import json
import os

INPUT_FILENAME = '../data/VanHoaVaDuLichVN.txt'
OUTPUT_FILENAME = '../data/VanHoaVaDuLichVN.json'

def clean_text(text):
    """Làm sạch văn bản OCR cơ bản"""
    if not text: return ""
    # Xóa dòng chỉ chứa số trang
    text = re.sub(r'\n\s*\d+\s*\n', '\n', text)
    # Xóa dấu chấm dẫn mục lục
    text = re.sub(r'\.{4,}\d*', '', text)
    # Nối dòng bị ngắt giữa chừng (giữ lại đoạn văn)
    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        line = line.strip()
        if not line: continue
        # Logic nối dòng: Dòng trước không kết thúc bằng dấu câu và dòng này không viết hoa
        if cleaned_lines and not cleaned_lines[-1].endswith(('.', ':', '!', '?')) and not line[0].isupper():
            cleaned_lines[-1] += " " + line
        else:
            cleaned_lines.append(line)
    return "\n".join(cleaned_lines)

def parse_toc_structure(content):
    """
    Phân tích Mục Lục để lấy danh sách Tỉnh và Địa danh chi tiết
    """
    print("🔍 Đang phân tích Mục Lục...")
    
    # 1. Tìm vùng Mục lục
    start_marker = "BẢNG TRA NHANH CÁC TỈNH"
    start_idx = content.rfind(start_marker)
    if start_idx == -1: 
        # Fallback
        start_idx = content.rfind("BẢNG TRA NHANH")
    
    if start_idx == -1:
        print("❌ Không tìm thấy mục lục bảng tra. Kiểm tra lại file.")
        return [], []

    toc_content = content[start_idx:]
    
    # 2. Tách phần Tỉnh và phần Di tích
    # Dựa vào từ khóa ngăn cách trong sách
    split_marker = "BẢNG TRA CÁC THẮNG CẢNH"
    parts = toc_content.split(split_marker)
    
    toc_provinces_text = parts[0]
    toc_attractions_text = parts[1] if len(parts) > 1 else ""

    # Regex bắt dòng: Tên ... Số trang
    line_pattern = re.compile(r'^(.+?)(?:\.{3,})\s*\d+$', re.MULTILINE)

    # Danh sách Tỉnh
    provinces = []
    for m in line_pattern.findall(toc_provinces_text):
        name = m.strip()
        if len(name) > 3 and "BẢNG TRA" not in name:
            provinces.append(name)

    # Danh sách Chi tiết (Thắng cảnh/Di tích)
    attractions = []
    for m in line_pattern.findall(toc_attractions_text):
        name = m.strip()
        if len(name) > 3:
            attractions.append(name)

    # Sắp xếp theo độ dài giảm dần để regex ưu tiên chuỗi dài
    provinces = sorted(list(set(provinces)), key=len, reverse=True)
    attractions = sorted(list(set(attractions)), key=len, reverse=True)
    
    print(f"✅ Đã tìm thấy: {len(provinces)} Tỉnh và {len(attractions)} Địa danh chi tiết.")
    return provinces, attractions

def extract_general_info(content, first_province):
    """Lấy phần đầu sách (trước khi vào tỉnh đầu tiên)"""
    print("📖 Đang lấy phần Đại Cương (Đầu sách)...")
    # Tìm vị trí tỉnh đầu tiên (thường là AN GIANG)
    match = re.search(rf'\n\s*{re.escape(first_province)}\s*\n', content)
    if not match:
        match = re.search(rf'{re.escape(first_province)}', content)
    
    end_idx = match.start() if match else 0
    general_text = content[:end_idx]
    
    # Chia nhỏ phần đại cương theo các chương lớn (Hardcode các chương phổ biến)
    chapters = [
        "LÃNH THỔ VIỆT NAM", "LỊCH SỬ VIỆT NAM", "CÁC DÂN TỘC TRÊN ĐẤT NƯỚC VIỆT NAM",
        "TÔN GIÁO - TÍN NGƯỠNG", "PHONG TỤC TẬP QUÁN", "LỄ HỘI TRUYỀN THỐNG", "ĂN MẶC CỦA NGƯỜI VIỆT"
    ]
    
    data = []
    # Cắt text theo chương
    indices = []
    for chap in chapters:
        m = re.search(rf'(?:\n|^)\s*{re.escape(chap)}', general_text)
        if m: indices.append({"name": chap, "start": m.start()})
    
    indices.sort(key=lambda x: x["start"])
    
    for i, item in enumerate(indices):
        name = item["name"]
        start = item["start"]
        end = indices[i+1]["start"] if i+1 < len(indices) else len(general_text)
        
        content_chap = clean_text(general_text[start:end])
        data.append({
            "id": f"GEN_{i:02d}",
            "name": name,
            "type": "GeneralTopic",
            "content": content_chap,
            "embedding_text": f"Tổng quan Việt Nam - {name}: {content_chap[:500]}"
        })
        
    return data

def extract_all_content(file_path):
    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        return

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # --- FIX 1: CẮT BỎ HOÀN TOÀN PHẦN MỤC LỤC CUỐI FILE TRƯỚC KHI XỬ LÝ ---
    # Tìm điểm bắt đầu của Bảng tra nhanh để loại bỏ nó khỏi phần nội dung chính
    toc_marker = "BẢNG TRA NHANH CÁC TỈNH"
    toc_idx = content.rfind(toc_marker)
    if toc_idx == -1: toc_idx = content.rfind("BẢNG TRA NHANH")
    
    # Body content chỉ lấy đến trước mục lục. 
    # Phần mục lục (toc_part) chỉ dùng để lấy danh sách tên, không dùng để quét nội dung.
    body_content = content[:toc_idx] if toc_idx != -1 else content
    toc_part = content[toc_idx:] if toc_idx != -1 else content

    # 1. Parse Mục Lục (Vẫn parse từ phần toc_part để lấy danh sách)
    provinces, attractions = parse_toc_structure(content) # Hàm này giữ nguyên logic cũ
    if not provinces: return

    final_dataset = {
        "General_Information": extract_general_info(body_content, provinces[0]), 
        "Provinces": []
    }

    # 2. Cắt nội dung theo từng Tỉnh (Dùng body_content đã sạch)
    print("🚀 Đang xử lý chi tiết từng tỉnh (Đã Fix lỗi Yên Bái)...")
    
    prov_markers = []
    for p in provinces:
        # Regex tìm tên tỉnh trong body_content
        m = re.search(rf'(?:\n|^)\s*{re.escape(p)}\s*(?:\n|$)', body_content)
        if m: prov_markers.append({"name": p, "start": m.start()})
    
    prov_markers.sort(key=lambda x: x["start"])

    for i, p_mk in enumerate(prov_markers):
        p_name = p_mk["name"]
        start = p_mk["start"]
        # Điểm cuối của tỉnh này là điểm bắt đầu của tỉnh sau.
        # Với tỉnh cuối cùng (Yên Bái), điểm cuối chính là len(body_content) (đã cắt mục lục)
        end = prov_markers[i+1]["start"] if i+1 < len(prov_markers) else len(body_content)
        
        prov_text = body_content[start:end]
        
        province_obj = {
            "id": f"P_{i:03d}",
            "name": p_name,
            "items": []
        }

        # 3. Quét địa danh trong text của tỉnh
        found_attrs = []
        for attr in attractions:
            # FIX 2: Regex chặt chẽ hơn để tránh bắt nhầm Header
            # Chỉ bắt nếu theo sau không phải là dòng chỉ chứa số trang
            am = re.search(rf'(?:\n|^)\s*(?:\d+\.\s*)?{re.escape(attr)}\s*(?:\n|\.\.\.)', prov_text, re.IGNORECASE)
            if am:
                found_attrs.append({"name": attr, "start": am.start(), "end_header": am.end()})
        
        found_attrs.sort(key=lambda x: x["start"])
        
        if not found_attrs:
            content_clean = clean_text(prov_text.replace(p_name, ""))
            # Chỉ thêm nếu nội dung không phải là header rác
            if len(content_clean) > 50 and "THẮNG CẢNH" not in content_clean:
                province_obj["items"].append({
                    "name": f"Tổng quan {p_name}",
                    "type": "General",
                    "content": content_clean,
                    "embedding_text": f"Tổng quan về tỉnh {p_name}. {content_clean[:300]}"
                })
        else:
            # Lấy Intro
            intro_text = prov_text[:found_attrs[0]["start"]].replace(p_name, "")
            intro_clean = clean_text(intro_text)
            if len(intro_clean) > 50:
                 province_obj["items"].append({
                    "name": f"Giới thiệu chung {p_name}",
                    "type": "General",
                    "content": intro_clean,
                    "embedding_text": f"Tổng quan về tỉnh {p_name}. {intro_clean[:300]}"
                })
            
            # Lấy từng địa danh
            for k, attr in enumerate(found_attrs):
                a_name = attr["name"]
                a_start = attr["end_header"]
                a_end = found_attrs[k+1]["start"] if k+1 < len(found_attrs) else len(prov_text)
                
                raw_content = prov_text[a_start:a_end]
                
                # FIX 3: Loại bỏ các dòng Header thừa bị dính vào nội dung
                # Ví dụ: Loại bỏ dòng "THẮNG CẢNH, DI TÍCH VÀ LỄ HỘI" nếu nó nằm trong content
                raw_content = re.sub(r'THẮNG CẢNH, DI TÍCH VÀ LỄ HỘI', '', raw_content)
                
                a_content = clean_text(raw_content)
                
                # Chỉ lấy nếu nội dung đủ dài (tránh lấy nhầm dòng tiêu đề)
                if len(a_content) > 30: 
                    label = "Place"
                    if any(x in a_name.upper() for x in ["LỄ", "HỘI"]): label = "Festival"
                    elif any(x in a_name.upper() for x in ["CHÙA", "ĐỀN", "ĐÌNH", "MIẾU", "NHÀ THỜ"]): label = "Religious"
                    elif any(x in a_name.upper() for x in ["NÚI", "SÔNG", "HỒ", "THÁC", "RỪNG", "HANG", "ĐỘNG"]): label = "Nature"
                    elif any(x in a_name.upper() for x in ["BẢO TÀNG", "DI TÍCH"]): label = "History"
                    
                    province_obj["items"].append({
                        "name": a_name,
                        "type": label,
                        "content": a_content,
                        "embedding_text": f"{a_name} ({label}) tại {p_name}. {a_content[:300]}"
                    })

        final_dataset["Provinces"].append(province_obj)
        print(f"  -> {p_name}: {len(province_obj['items'])} mục.")

    with open(OUTPUT_FILENAME, 'w', encoding='utf-8') as f:
        json.dump(final_dataset, f, ensure_ascii=False, indent=2)
    
    print(f"\n✅ HOÀN TẤT FIX! File '{OUTPUT_FILENAME}' đã sạch lỗi Yên Bái.")
    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        return

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # 1. Parse Mục Lục
    provinces, attractions = parse_toc_structure(content)
    if not provinces: return

    final_dataset = {
        "General_Information": extract_general_info(content, provinces[0]), # Lấy phần đầu sách
        "Provinces": []
    }

    # 2. Cắt nội dung theo từng Tỉnh
    print("🚀 Đang xử lý chi tiết từng tỉnh...")
    
    # Tìm vị trí bắt đầu của mỗi tỉnh trong text
    prov_markers = []
    for p in provinces:
        # Regex tìm tên tỉnh đứng riêng dòng hoặc in hoa
        m = re.search(rf'(?:\n|^)\s*{re.escape(p)}\s*(?:\n|$)', content)
        if m: prov_markers.append({"name": p, "start": m.start()})
    
    prov_markers.sort(key=lambda x: x["start"])

    for i, p_mk in enumerate(prov_markers):
        p_name = p_mk["name"]
        start = p_mk["start"]
        # Kết thúc là tỉnh tiếp theo hoặc đến phần Mục Lục
        end = prov_markers[i+1]["start"] if i+1 < len(prov_markers) else content.rfind("BẢNG TRA NHANH")
        
        prov_text = content[start:end]
        
        province_obj = {
            "id": f"P_{i:03d}",
            "name": p_name,
            "items": []
        }

        # 3. Trong mỗi tỉnh, quét tìm các Địa danh/Lễ hội từ danh sách Attractions
        found_attrs = []
        for attr in attractions:
            # Tìm địa danh trong text của tỉnh này
            # Cho phép số thứ tự: "1. Chùa Hương"
            am = re.search(rf'(?:\n|^)\s*(?:\d+\.\s*)?{re.escape(attr)}\s*(?:\n|\.\.\.)', prov_text, re.IGNORECASE)
            if am:
                found_attrs.append({"name": attr, "start": am.start(), "end_header": am.end()})
        
        found_attrs.sort(key=lambda x: x["start"])
        
        # 4. Cắt text cho từng địa danh tìm thấy
        if not found_attrs:
            # Nếu không tìm thấy địa danh nào, lấy toàn bộ text làm "Giới thiệu chung"
            province_obj["items"].append({
                "name": f"Tổng quan {p_name}",
                "type": "General",
                "content": clean_text(prov_text.replace(p_name, ""))
            })
        else:
            # Lấy phần đầu (Intro)
            intro_text = prov_text[:found_attrs[0]["start"]].replace(p_name, "")
            if len(intro_text.strip()) > 50:
                province_obj["items"].append({
                    "name": f"Giới thiệu chung {p_name}",
                    "type": "General",
                    "content": clean_text(intro_text),
                    "embedding_text": f"Tổng quan về tỉnh {p_name}. {intro_text[:300]}"
                })
            
            # Lấy từng địa danh
            for k, attr in enumerate(found_attrs):
                a_name = attr["name"]
                a_start = attr["end_header"]
                a_end = found_attrs[k+1]["start"] if k+1 < len(found_attrs) else len(prov_text)
                
                a_content = clean_text(prov_text[a_start:a_end])
                
                if len(a_content) > 20: # Bỏ qua rác
                    # Auto-Labeling đơn giản
                    label = "Place"
                    if any(x in a_name.upper() for x in ["LỄ", "HỘI"]): label = "Festival"
                    elif any(x in a_name.upper() for x in ["CHÙA", "ĐỀN", "ĐÌNH", "MIẾU"]): label = "Religious"
                    elif any(x in a_name.upper() for x in ["NÚI", "SÔNG", "HỒ", "THÁC", "RỪNG"]): label = "Nature"
                    
                    province_obj["items"].append({
                        "name": a_name,
                        "type": label,
                        "content": a_content,
                        "embedding_text": f"{a_name} ({label}) tại {p_name}. {a_content[:300]}"
                    })

        final_dataset["Provinces"].append(province_obj)
        print(f"  -> {p_name}: {len(province_obj['items'])} mục.")

    # Xuất file JSON
    with open(OUTPUT_FILENAME, 'w', encoding='utf-8') as f:
        json.dump(final_dataset, f, ensure_ascii=False, indent=2)
    
    print(f"\n🎉 HOÀN TẤT! Đã xuất toàn bộ dữ liệu ra '{OUTPUT_FILENAME}'")
    
    # Thống kê
    total_items = sum(len(p['items']) for p in final_dataset['Provinces']) + len(final_dataset['General_Information'])
    print(f"📊 Tổng cộng: {len(final_dataset['Provinces'])} Tỉnh và {total_items} Node dữ liệu.")

if __name__ == "__main__":
    extract_all_content(INPUT_FILENAME)

🔍 Đang phân tích Mục Lục...
✅ Đã tìm thấy: 64 Tỉnh và 1536 Địa danh chi tiết.
📖 Đang lấy phần Đại Cương (Đầu sách)...
🚀 Đang xử lý chi tiết từng tỉnh (Đã Fix lỗi Yên Bái)...
  -> HÀ NỘI: 178 mục.
  -> AN GIANG: 17 mục.
  -> BÀ RỊA-VŨNG TÀU: 33 mục.
  -> BẠC LIÊU: 12 mục.
  -> BẮC GIANG: 26 mục.
  -> BẮC KẠN: 22 mục.
  -> BẮC NINH: 19 mục.
  -> BẾN TRE: 19 mục.
  -> BÌNH DƯƠNG: 15 mục.
  -> BÌNH ĐỊNH: 24 mục.
  -> BÌNH PHƯỚC: 13 mục.
  -> BÌNH THUẬN: 32 mục.
  -> CÀ MAU: 16 mục.
  -> CAO BẰNG: 30 mục.
  -> THÀNH PHỐ CẦN THƠ: 14 mục.
  -> THÀNH PHỐ ĐÀ NẴNG: 21 mục.
  -> ĐẮK LẮK: 14 mục.
  -> ĐẮK NÔNG: 16 mục.
  -> ĐIỆN BIÊN: 11 mục.
  -> ĐỒNG NAI: 18 mục.
  -> ĐỒNG THÁP: 18 mục.
  -> GIA LAI: 18 mục.
  -> HÀ GIANG: 25 mục.
  -> HÀ NAM: 54 mục.
  -> HÀ TÂY: 25 mục.
  -> HÀ TĨNH: 23 mục.
  -> HẢI DƯƠNG: 22 mục.
  -> HẢI PHÒNG: 19 mục.
  -> HẬU GIANG: 7 mục.
  -> HÒA BÌNH: 15 mục.
  -> THÀNH PHỐ HỒ CHÍ MINH: 48 mục.
  -> HƯNG YÊN: 38 mục.
  -> KIÊN GIANG: 27 mục.
  -> KON TUM: 13 mục.
  -> 

### => Vẫn tồn đọng lỗi trên, chưa triệt để


### Tiếp tục fix lỗi

In [4]:
import re
import json
import os

# Tên file của bạn
INPUT_FILENAME = '../data/VanHoaVaDuLichVN.txt'
OUTPUT_FILENAME = '../data/VanHoaVaDuLichVN.json'

def clean_text(text):
    """Làm sạch văn bản OCR"""
    if not text: return ""
    text = re.sub(r'\n\s*\d+\s*\n', '\n', text) # Xóa số trang lẻ loi
    text = re.sub(r'\.{4,}\d*', '', text) # Xóa chấm mục lục
    # Nối dòng bị ngắt
    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        line = line.strip()
        if not line: continue
        # Nếu dòng trước không kết thúc bằng dấu câu và dòng này không viết hoa -> Nối
        if cleaned_lines and not cleaned_lines[-1].endswith(('.', ':', '!', '?')) and not line[0].isupper():
            cleaned_lines[-1] += " " + line
        else:
            cleaned_lines.append(line)
    return "\n".join(cleaned_lines)

def parse_toc_structure(content):
    """Lấy danh sách Tỉnh và Địa danh từ Mục Lục"""
    print("🔍 Đang phân tích Mục Lục...")
    
    # Tìm vùng Mục lục ở cuối sách
    start_marker = "BẢNG TRA NHANH CÁC TỈNH"
    start_idx = content.rfind(start_marker)
    if start_idx == -1: start_idx = content.rfind("BẢNG TRA NHANH")
    
    if start_idx == -1:
        print("❌ Lỗi: Không tìm thấy Mục Lục. Hãy kiểm tra lại file text.")
        return [], []

    toc_content = content[start_idx:]
    
    # Tách phần Tỉnh và phần Thắng cảnh
    parts = toc_content.split("BẢNG TRA CÁC THẮNG CẢNH")
    toc_provinces_text = parts[0]
    toc_attractions_text = parts[1] if len(parts) > 1 else ""

    line_pattern = re.compile(r'^(.+?)(?:\.{3,})\s*\d+$', re.MULTILINE)

    provinces = [m.strip() for m in line_pattern.findall(toc_provinces_text) if len(m.strip()) > 3 and "BẢNG TRA" not in m]
    attractions = [m.strip() for m in line_pattern.findall(toc_attractions_text) if len(m.strip()) > 3]

    # Sắp xếp độ dài giảm dần để khớp chính xác
    provinces = sorted(list(set(provinces)), key=len, reverse=True)
    attractions = sorted(list(set(attractions)), key=len, reverse=True)
    
    return provinces, attractions

def extract_general_info(content, first_province):
    """Lấy phần Đại Cương đầu sách"""
    print("📖 Đang lấy phần Đại Cương...")
    match = re.search(rf'\n\s*{re.escape(first_province)}\s*\n', content)
    end_idx = match.start() if match else 0
    general_text = content[:end_idx]
    
    chapters = ["LÃNH THỔ VIỆT NAM", "LỊCH SỬ VIỆT NAM", "CÁC DÂN TỘC TRÊN ĐẤT NƯỚC VIỆT NAM",
                "TÔN GIÁO - TÍN NGƯỠNG", "PHONG TỤC TẬP QUÁN", "LỄ HỘI TRUYỀN THỐNG", "ĂN MẶC CỦA NGƯỜI VIỆT"]
    
    data = []
    indices = []
    for chap in chapters:
        m = re.search(rf'(?:\n|^)\s*{re.escape(chap)}', general_text)
        if m: indices.append({"name": chap, "start": m.start()})
    indices.sort(key=lambda x: x["start"])
    
    for i, item in enumerate(indices):
        name = item["name"]
        start = item["start"]
        end = indices[i+1]["start"] if i+1 < len(indices) else len(general_text)
        content_chap = clean_text(general_text[start:end])
        data.append({
            "id": f"GEN_{i:02d}", "name": name, "type": "GeneralTopic",
            "content": content_chap,
            "embedding_text": f"Tổng quan Văn hóa Việt Nam - {name}: {content_chap[:300]}"
        })
    return data

def extract_all_content_fixed(file_path):
    if not os.path.exists(file_path):
        print("❌ Không tìm thấy file.")
        return

    with open(file_path, 'r', encoding='utf-8') as f:
        full_content = f.read()

    # --- QUAN TRỌNG: CẮT BỎ MỤC LỤC KHỎI NỘI DUNG ---
    toc_marker = "BẢNG TRA NHANH CÁC TỈNH"
    toc_idx = full_content.rfind(toc_marker)
    if toc_idx == -1: toc_idx = full_content.rfind("BẢNG TRA NHANH")
    
    # Body content chỉ lấy đến trước mục lục -> Khắc phục lỗi Yên Bái bị lẫn
    body_content = full_content[:toc_idx] if toc_idx != -1 else full_content

    # 1. Lấy danh sách từ Mục Lục (vẫn dùng full_content để quét danh sách)
    provinces, attractions = parse_toc_structure(full_content)
    if not provinces: return

    final_dataset = {
        "General_Information": extract_general_info(body_content, provinces[0]),
        "Provinces": []
    }

    # 2. Xử lý từng Tỉnh
    print("🚀 Đang xử lý chi tiết (Đã fix lỗi Yên Bái & Header)...")
    
    prov_markers = []
    for p in provinces:
        m = re.search(rf'(?:\n|^)\s*{re.escape(p)}\s*(?:\n|$)', body_content)
        if m: prov_markers.append({"name": p, "start": m.start()})
    prov_markers.sort(key=lambda x: x["start"])

    for i, p_mk in enumerate(prov_markers):
        p_name = p_mk["name"]
        start = p_mk["start"]
        # Điểm cuối là tỉnh tiếp theo hoặc HẾT body_content
        end = prov_markers[i+1]["start"] if i+1 < len(prov_markers) else len(body_content)
        
        prov_text = body_content[start:end]
        
        province_obj = { "id": f"P_{i:03d}", "name": p_name, "items": [] }

        # Tìm địa danh trong tỉnh này
        found_attrs = []
        for attr in attractions:
            am = re.search(rf'(?:\n|^)\s*(?:\d+\.\s*)?{re.escape(attr)}\s*(?:\n|\.\.\.)', prov_text, re.IGNORECASE)
            if am: found_attrs.append({"name": attr, "start": am.start(), "end_header": am.end()})
        found_attrs.sort(key=lambda x: x["start"])
        
        # Nếu không có địa danh -> Lấy hết làm Tổng quan
        if not found_attrs:
            clean_c = clean_text(prov_text.replace(p_name, ""))
            # Lọc rác header
            clean_c = re.sub(r'THẮNG CẢNH, DI TÍCH VÀ LỄ HỘI', '', clean_c).strip()
            
            if len(clean_c) > 50:
                province_obj["items"].append({
                    "name": f"Tổng quan {p_name}", "type": "General",
                    "content": clean_c,
                    "embedding_text": f"Tổng quan về tỉnh {p_name}. {clean_c[:300]}"
                })
        else:
            # Lấy Intro
            intro_raw = prov_text[:found_attrs[0]["start"]].replace(p_name, "")
            intro_clean = clean_text(re.sub(r'THẮNG CẢNH, DI TÍCH VÀ LỄ HỘI', '', intro_raw))
            if len(intro_clean) > 50:
                 province_obj["items"].append({
                    "name": f"Giới thiệu chung {p_name}", "type": "General",
                    "content": intro_clean,
                    "embedding_text": f"Tổng quan về tỉnh {p_name}. {intro_clean[:300]}"
                })
            
            # Lấy từng địa danh
            for k, attr in enumerate(found_attrs):
                a_name = attr["name"]
                a_start = attr["end_header"]
                a_end = found_attrs[k+1]["start"] if k+1 < len(found_attrs) else len(prov_text)
                
                raw_content = prov_text[a_start:a_end]
                # --- QUAN TRỌNG: XÓA HEADER RÁC ---
                raw_content = re.sub(r'THẮNG CẢNH, DI TÍCH VÀ LỄ HỘI', '', raw_content)
                a_content = clean_text(raw_content)
                
                # Chỉ lấy nếu nội dung thực sự dài (tránh lấy nhầm tiêu đề rỗng)
                if len(a_content) > 40: 
                    label = "Place"
                    if any(x in a_name.upper() for x in ["LỄ", "HỘI"]): label = "Festival"
                    elif any(x in a_name.upper() for x in ["CHÙA", "ĐỀN", "ĐÌNH", "MIẾU", "NHÀ THỜ"]): label = "Religious"
                    elif any(x in a_name.upper() for x in ["NÚI", "SÔNG", "HỒ", "THÁC", "RỪNG", "HANG", "ĐỘNG"]): label = "Nature"
                    
                    province_obj["items"].append({
                        "name": a_name, "type": label,
                        "content": a_content,
                        "embedding_text": f"{a_name} ({label}) tại {p_name}. {a_content[:300]}"
                    })

        final_dataset["Provinces"].append(province_obj)
        print(f"  -> {p_name}: {len(province_obj['items'])} mục.")

    with open(OUTPUT_FILENAME, 'w', encoding='utf-8') as f:
        json.dump(final_dataset, f, ensure_ascii=False, indent=2)
    
    print(f"\n✅ ĐÃ XONG! File '{OUTPUT_FILENAME}' đã sạch 100%.")

if __name__ == "__main__":
    extract_all_content_fixed(INPUT_FILENAME)

🔍 Đang phân tích Mục Lục...
📖 Đang lấy phần Đại Cương...
🚀 Đang xử lý chi tiết (Đã fix lỗi Yên Bái & Header)...
  -> HÀ NỘI: 178 mục.
  -> AN GIANG: 17 mục.
  -> BÀ RỊA-VŨNG TÀU: 33 mục.
  -> BẠC LIÊU: 12 mục.
  -> BẮC GIANG: 26 mục.
  -> BẮC KẠN: 22 mục.
  -> BẮC NINH: 19 mục.
  -> BẾN TRE: 19 mục.
  -> BÌNH DƯƠNG: 15 mục.
  -> BÌNH ĐỊNH: 24 mục.
  -> BÌNH PHƯỚC: 13 mục.
  -> BÌNH THUẬN: 32 mục.
  -> CÀ MAU: 16 mục.
  -> CAO BẰNG: 30 mục.
  -> THÀNH PHỐ CẦN THƠ: 14 mục.
  -> THÀNH PHỐ ĐÀ NẴNG: 21 mục.
  -> ĐẮK LẮK: 14 mục.
  -> ĐẮK NÔNG: 16 mục.
  -> ĐIỆN BIÊN: 11 mục.
  -> ĐỒNG NAI: 18 mục.
  -> ĐỒNG THÁP: 18 mục.
  -> GIA LAI: 18 mục.
  -> HÀ GIANG: 25 mục.
  -> HÀ NAM: 54 mục.
  -> HÀ TÂY: 25 mục.
  -> HÀ TĨNH: 23 mục.
  -> HẢI DƯƠNG: 22 mục.
  -> HẢI PHÒNG: 19 mục.
  -> HẬU GIANG: 7 mục.
  -> HÒA BÌNH: 15 mục.
  -> THÀNH PHỐ HỒ CHÍ MINH: 48 mục.
  -> HƯNG YÊN: 38 mục.
  -> KIÊN GIANG: 27 mục.
  -> KON TUM: 13 mục.
  -> LAI CHÂU: 6 mục.
  -> LẠNG SƠN: 29 mục.
  -> LÀO CAI: 23 mục.
